[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sunfish256/get-weather-from-jma/blob/main/src/get_weather.ipynb)

<span style="font-size: 200%">**日ごとの過去気象データを取得する関数**</span>  

このスクリプトでは東京（気象台）を例とする。  
その他の観測地点の気象データを取得する場合は、
[気象庁｜過去の気象データ検索](https://www.data.jma.go.jp/obd/stats/etrn/index.php?prec_no=44&block_no=47662&year=2019&month=02&day=01&view=p1)
にアクセスし、「地点の選択」を実行する。  
その後、URLから位置情報（**prec_no**と**block_no**）を取得して、loc_info引数を書き変える。  
また、prec_noの直前が**daily_<font color="Red">s</font>1.php**（気象台・観測所データ）と**daily_<font color="Red">a</font>1.php**（アメダスデータ）のどちらであるかも要注意。  
アメダスデータの場合、コメントアウトされている最終セルを代わりに実行する。  

観測地点によっては観測項目の違いがあるかもしれないため、エラーの際は
[気象庁｜過去の気象データ検索｜日ごとの値](https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2023&month=1&day=01&view=p1)
（リンク先は東京の値）の表の項目が一致しているかを確認する。  
一致していなければscraping関数のdata_list.append部分と、create_csv関数のfields変数を書き換える。

In [ ]:
import csv
import datetime
import re
import urllib.request
import numpy as np
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

入力項目

In [ ]:
# 出力先パス
csv_path = 'output_sample.csv'
# データ取得開始・終了日
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date.today()
# 観測地点のURL情報
loc_info = 'daily_s1.php?prec_no=44&block_no=47662&'

In [ ]:
def str2float(weather_data):
    if weather_data is None:
        return np.nan
    # 表データ内に"])半角スペース"が混入するケースがあったので除去
    else:
        value = re.sub(r'[\]) ]', '', weather_data)
    try:
        return float(value)
    except:
        if value == '--':
            return 0
        if value == '///':
            return np.nan
        else:
            raise ValueError(f'Unexpected value "{weather_data}".')


def str2str(weather_data):
    if weather_data is None:
        return np.nan
    # 表データ内に"])半角スペース"が混入するケースがあったので除去
    else:
        value = re.sub(r'[\]) ]', '', weather_data)
    if value == '--':
        return 0
    if value == '///':
        return np.nan
    else:
        return value


def scraping(url, date):

    # 気象データのページを取得
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html)
    trs = soup.find('table', { 'class' : 'data2_s' })

    data_list = []
    data_list_per_month = []

    # table の中身を取得
    for tr in trs.findAll('tr'):
        tds = tr.findAll('td')

        if tds:
            data_list.append(f'{date.year}-{date.month}-{tds[0].string}')
            data_list.append(str2float(tds[1].string))
            data_list.append(str2float(tds[2].string))
            data_list.append(str2float(tds[3].string))
            data_list.append(str2float(tds[4].string))
            data_list.append(str2float(tds[5].string))
            data_list.append(str2float(tds[6].string))
            data_list.append(str2float(tds[7].string))
            data_list.append(str2float(tds[8].string))
            data_list.append(str2float(tds[9].string))
            data_list.append(str2float(tds[10].string))
            data_list.append(str2float(tds[11].string))
            data_list.append(str2float(tds[12].string))
            data_list.append(str2str(tds[13].string))
            data_list.append(str2float(tds[14].string))
            data_list.append(str2str(tds[15].string))
            data_list.append(str2float(tds[16].string))
            data_list.append(str2float(tds[17].string))
            data_list.append(str2float(tds[18].string))
            data_list.append(str2str(tds[19].string))
            data_list.append(str2str(tds[20].string))

            data_list_per_month.append(data_list)
            data_list = []

    return data_list_per_month


def create_csv(csv_path, start_date, end_date, loc_info):
    # CSV の列
    fields = [
        '年月日',
        '気圧_現地', '気圧_海面',
        '降水量_合計', '降水量_1時間最大', '降水量_10分間最大',
        '気温_平均', '気温_最高', '気温_最低',
        '湿度_平均', '湿度_最小',
        '風速_平均', '風速_最大', '風向_最大', '風速_最大瞬間', '風向_最大瞬間',
        '日照時間',
        '降雪_合計', '積雪_最深',
        '天気_昼', '天気_夜'
    ]

    with open(csv_path, 'w', encoding='utf-8-sig') as f:
        writer = csv.writer(f, lineterminator='\n')
        writer.writerow(fields)

        date = start_date
        while date <= end_date:
            print(f'Getiing data for {date}')
            # URLを指定：観測地点の日ごとの気象データ（月別）
            url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/' \
                  + loc_info + \
                  f'year={date.year}&month={date.month}&day=&view='

            data_list_per_month = scraping(url, date)

            for d in data_list_per_month:
                writer.writerow(d)

            date += relativedelta(months=1)
            
    print('Complete!')


if __name__ == '__main__':
    create_csv(csv_path, start_date, end_date, loc_info)

<br>**以下、アメダスの場合**

In [ ]:
# def str2float(weather_data):
#     if weather_data is None:
#         return np.nan
#     # 表データ内に"])半角スペース"が混入するケースがあったので除去
#     else:
#         value = re.sub(r'[\]) ]', '', weather_data)
#     try:
#         return float(value)
#     except:
#         if value == '--':
#             return 0
#         if value == '///':
#             return np.nan
#         else:
#             raise ValueError(f'Unexpected value "{weather_data}".')


# def str2str(weather_data):
#     if weather_data is None:
#         return np.nan
#     # 表データ内に"])半角スペース"が混入するケースがあったので除去
#     else:
#         value = re.sub(r'[\]) ]', '', weather_data)
#     if value == '--':
#         return 0
#     if value == '///':
#         return np.nan
#     else:
#         return value


# def scraping(url, date):

#     # 気象データのページを取得
#     html = urllib.request.urlopen(url).read()
#     soup = BeautifulSoup(html)
#     trs = soup.find('table', { 'class' : 'data2_s' })

#     data_list = []
#     data_list_per_month = []

#     # table の中身を取得
#     for tr in trs.findAll('tr'):
#         tds = tr.findAll('td')

#         if tds:
#             data_list.append(f'{date.year}-{date.month}-{tds[0].string}')
#             data_list.append(str2float(tds[1].string))
#             data_list.append(str2float(tds[2].string))
#             data_list.append(str2float(tds[3].string))
#             data_list.append(str2float(tds[4].string))
#             data_list.append(str2float(tds[5].string))
#             data_list.append(str2float(tds[6].string))
#             data_list.append(str2float(tds[7].string))
#             data_list.append(str2float(tds[8].string))
#             data_list.append(str2float(tds[9].string))
#             data_list.append(str2float(tds[10].string))
#             data_list.append(str2str(tds[11].string))
#             data_list.append(str2float(tds[12].string))
#             data_list.append(str2str(tds[13].string))
#             data_list.append(str2str(tds[14].string))
#             data_list.append(str2float(tds[15].string))
#             data_list.append(str2float(tds[16].string))
#             data_list.append(str2float(tds[17].string))

#             data_list_per_month.append(data_list)
#             data_list = []

#     return data_list_per_month


# def create_csv(csv_path, start_date, end_date, loc_info):
#     # CSV の列
#     fields = [
#         '年月日',
#         '降水量_合計', '降水量_最大_60min', '降水量_最大_10min',
#         '気温_平均', '気温_最高', '気温_最低',
#         '湿度_平均', '湿度_最小',
#         '風速_平均', '風速_最大', '風向_最大', '風速_最大瞬間', '風向_最大瞬間', '最多風向',
#         '日照時間',
#         '降雪_合計', '積雪_最深',
#     ]

#     with open(csv_path, 'w', encoding='utf-8-sig') as f:
#         writer = csv.writer(f, lineterminator='\n')
#         writer.writerow(fields)

#         date = start_date
#         while date <= end_date:
#             print(f'Getiing data for {date}')
#             # URLを指定：観測地点の日ごとの気象データ（月別）
#             url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/' \
#                   + loc_info + \
#                   f'year={date.year}&month={date.month}&day=&view='

#             data_list_per_month = scraping(url, date)

#             for d in data_list_per_month:
#                 writer.writerow(d)

#             date += relativedelta(months=1)

#     print('Complete!')


# if __name__ == '__main__':
#     create_csv(csv_path, start_date, end_date, loc_info)